In [1]:
import os
import numpy as np
import pandas as pd
from collections import deque

!pip install music21
!pip install pygame

from music21 import converter, instrument, note, chord, midi, stream, meter

from keras.models import Model, Sequential
from keras.layers import *
from keras.models import load_model
import keras.backend as K
import keras.callbacks
from keras.utils import Sequence
from keras.utils import to_categorical
from keras.preprocessing.sequence import TimeseriesGenerator

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Using TensorFlow backend.


In [2]:
# grab / parse data

In [3]:
directory = "midi/Piano"
# specify the number of track in each song (should be the same, in same order, for each song)
tracks = (0,)
instru = instrument.Piano()

notes = [[] for track in tracks]
durations = [[] for track in tracks]
for i, file in enumerate(os.listdir(directory)):
    midi_part = converter.parse(os.path.join(directory, file))
    # Parse the midi file by the notes/chords it contains
    for track, _ in enumerate(tracks):
        notes_to_parse = midi_part[tracks[track]].flat.notesAndRests
        notes_to_parse = notes_to_parse.makeMeasures()
        for mes in notes_to_parse.measures(0, None):
            if isinstance(mes, stream.Measure):
                #mes = mes.chordify()
                notes_mes_tab = []
                durations_mes_tab = []
                for elem in mes:    
                    if isinstance(elem, note.Note):
                        notes_mes_tab.append([str(elem.pitch)])
                    elif isinstance(elem, chord.Chord):
                        notes_mes_tab.append([str(n.nameWithOctave) for n in elem.pitches])
                    elif isinstance(elem, note.Rest):
                        notes_mes_tab.append([elem.name])
                    durations_mes_tab.append(elem.quarterLength)
                notes[track].append(notes_mes_tab)
                durations[track].append(durations_mes_tab)
    print("Song %s Loaded" % file)
print("DONE LOADING SONGS") 
# Get all notes
np_notes = [[] for track in tracks]
for track, _ in enumerate(tracks):
    for elem in notes[track]:
        np_notes[track] += elem
np_notes = [np.array(np_notes[track]) for track in tracks]
np_notes = [np_notes[track].flatten() for track in tracks]
notes_vocab = [[] for track in tracks]
for track, _ in enumerate(tracks):
    for item in np_notes[track]:
        if len(item) > 1 and item != "rest":
            notes_vocab[track].append(",".join([note for note in item]))
        else:
            notes_vocab[track].append(item[0])
    notes_vocab[track] = sorted(set(notes_vocab[track]))
# Get all durations
np_durations = [[] for track in tracks]
for track, _ in enumerate(tracks):
    for elem in durations[track]:
        np_durations[track] += elem
durations_vocab = [[] for track in tracks]
for track, _ in enumerate(tracks):
    durations_vocab[track] = sorted(set(np.array(np_durations[track]).flatten()))

for track, _ in enumerate(tracks):
    print("%s Measures, notes_Vocab : %s ; durations_Vocab : %s" % (len(notes[track]), len(notes_vocab[track]), len(durations_vocab[track])))
print(notes_vocab, notes)
print(durations_vocab, durations)

Song br_im2_format0.mid Loaded
Song br_im5_format0.mid Loaded
Song br_im6_format0.mid Loaded
Song br_rhap_format0.mid Loaded
Song chpn_op7_1_format0.mid Loaded
Song chpn_op7_2_format0.mid Loaded
Song chp_op18_format0.mid Loaded
Song chp_op31_format0.mid Loaded
Song Etude op10 n01.mid Loaded
Song Etude op10 n02.mid Loaded
Song Etude op10 n10.mid Loaded
Song jsb2pin1.mid Loaded
Song jsb2pin4.mid Loaded
Song jsb2pin8.mid Loaded
DONE LOADING SONGS
1283 Measures, notes_Vocab : 2159 ; durations_Vocab : 103
[['A1', 'A1,A2', 'A1,A5,E5', 'A2', 'A2,A1', 'A2,A1,C#6', 'A2,A1,C#6,A5,E5', 'A2,A3', 'A2,A4,E4,C4', 'A2,A5,E5,C#6', 'A2,B-2,B-3', 'A2,C3,E3', 'A2,C4', 'A2,E-2', 'A2,E-3', 'A2,E-5,A4,C5', 'A2,E-5,C5,A4', 'A2,F5,D5,A4', 'A3', 'A3,A2', 'A3,A2,C#3', 'A3,A2,C4', 'A3,A4', 'A3,A4,A2', 'A3,A4,F#4', 'A3,A4,F#4,D1,D2', 'A3,B2', 'A3,B2,F#3', 'A3,B4,F#2,F#3,B2', 'A3,C#3', 'A3,C#4', 'A3,C#4,A4', 'A3,C#4,F#4', 'A3,C3', 'A3,C4,E3,E4', 'A3,C4,E4,A5,E5', 'A3,C4,F#4', 'A3,C4,F4', 'A3,D4,F#4', 'A3,E-3,E-4,G2

In [4]:
maxLen = 0
for track, _ in enumerate(tracks):
    currMax = max(len(elem) for elem in notes[track])
    if currMax > maxLen:
        maxLen = currMax
for track, _ in enumerate(tracks):
    currMax = max(len(elem) for elem in durations[track])
    if currMax > maxLen:
        maxLen = currMax
        
print(maxLen)

65


In [5]:
cat_notes = [[] for track in tracks]
cat_durations = [[] for track in tracks]

for track, _ in enumerate(tracks):
    for mes in notes[track]:
        cat_tab = []
        for elem in mes:
            int_note = notes_vocab[track].index(",".join(elem))
            cat = np.zeros((len(notes_vocab[track])))
            cat[int_note] = 1
            cat_tab.append(cat)
        while len(cat_tab) < maxLen:
            cat_tab.append(np.zeros((len(notes_vocab[track]))))
        cat_notes[track].append(cat_tab)
    for mes in durations[track]:
        cat_tab = []
        for elem in mes:
            int_duration = durations_vocab[track].index(elem)
            cat = np.zeros((len(durations_vocab[track])))
            cat[int_duration] = 1
            cat_tab.append(cat)
        while len(cat_tab) < maxLen:
            cat_tab.append(np.zeros((len(durations_vocab[track]))))
        cat_durations[track].append(cat_tab)
# merge
x = [cat_notes, cat_durations]

In [6]:
# Build generator

In [7]:
batch_size = 32
#split = int(0.8 * len(x))

class dataGenerator(Sequence):
    def __init__(self, x, batch_size):
        self.batch_size = batch_size
        self.x = x
    def __len__(self):
        return len(self.x[0][0]) // self.batch_size - 1
    def __getitem__(self, idx):
        X_note = [[] for track in tracks]
        X_duration = [[] for track in tracks]
        Y_note = [[] for track in tracks]
        Y_duration = [[] for track in tracks]
        res = [[], []]
        for track, _ in enumerate(tracks):
            for i in range(self.batch_size):
                X_note[track].append(self.x[0][track][idx + i])
                X_duration[track].append(self.x[1][track][idx + i])
                Y_note[track].append(self.x[0][track][idx + i + 1])
                Y_duration[track].append(self.x[1][track][idx + i + 1])
            X_note[track] = np.array(X_note[track])
            X_duration[track] = np.array(X_duration[track])
            Y_note[track] = np.array(Y_note[track])
            Y_duration[track] = np.array(Y_duration[track])
            res[0].append(X_note[track])
            res[0].append(X_duration[track])
            res[1].append(Y_note[track])
            res[1].append(Y_duration[track])
        return res
data_gen = dataGenerator(x, batch_size)

In [8]:
data_gen[0][0][0].shape

(32, 65, 2159)

In [ ]:
# Build Model

In [ ]:
K.clear_session()

inputs_list = []
tracks_list = []
outputs_list = []

for track, _ in enumerate(tracks):
    in_note = Input(shape=(maxLen, len(notes_vocab[track])))
    inputs_list.append(in_note)
    x1 = LSTM(32, return_sequences=True)(in_note)
    tracks_list.append(x1)
    in_duration = Input(shape=(maxLen, len(durations_vocab[track])))
    inputs_list.append(in_duration)
    x2 = LSTM(32, return_sequences=True)(in_duration)
    tracks_list.append(x2)

concat = Concatenate()(tracks_list)
shared = TimeDistributed(Dense(64, activation='relu'))(concat)

for track, _ in enumerate(tracks):
    out_note = Dense(len(notes_vocab[track]), activation='softmax')(shared)
    outputs_list.append(out_note)
    
    out_duration = Dense(len(durations_vocab[track]), activation='softmax')(shared)
    outputs_list.append(out_duration)

model = Model(inputs=inputs_list, 
              outputs=outputs_list)
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc'])

model.summary()

In [ ]:
#save best model if model improved
model_name = "Piano_measured.h5"
best_checkpoint = keras.callbacks.ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
h = model.fit_generator(data_gen, epochs = 100)

In [ ]:
#print History graph
historydf = pd.DataFrame(h.history, index=h.epoch)
historydf.plot(ylim=(0,1))

In [ ]:
# Load / save

In [ ]:
model_name = "Piano_measured.h5"
model.save(os.path.join("weight_save", model_name))
#model = load_model(os.path.join("weight_save", model_name))

In [434]:
# Test

In [11]:
# Build seed
idx = 0
x_test, _ = data_gen[idx]
x_test = [x_test[i][0] for i in range(len(x_test))]
x_test_stream = [stream.Stream() for track in tracks]

for track, _ in enumerate(tracks):
    for i in range(maxLen):
        if 1 in x_test[2*track][i]:
            str_note = notes_vocab[track][np.argmax(x_test[2*track][i])]
            duration = durations_vocab[track][np.argmax(x_test[2*track+1][i])]
            if len(str_note.split(",")) > 1:
                _chord = chord.Chord(str_note.split(","))
                _chord.quarterLength = duration
                x_test_stream[track].append(_chord)
            else:
                if str_note != "rest":
                    _note = note.Note(str_note)
                    _note.quarterLength = duration
                    x_test_stream[track].append(_note)
                else:
                    _rest = note.Rest()
                    _rest.quarterLength = duration
                    x_test_stream[track].append(_rest)
    x_test_stream[track].insert(0, instru)

In [12]:
# make prediction from seed
x = x_test
pred = [[] for track in tracks]
preds = [[] for track in tracks]

for track, _ in enumerate(tracks):
    for mes in range(4):        
        pred[track] = model.predict([np.array([x[i]]) for i in range(len(x))])
        x = [pred[track][i][0] for i in range(len(pred[track]))]
        preds[track].append(pred[track])


NameError: name 'model' is not defined

In [441]:
# Build predicted stream
y_test_stream = [stream.Stream() for track in tracks]

for track, _ in enumerate(tracks):
    for j in range(len(preds[track])):
        for i in range(maxLen):
            if 1 in x_test[2*track][i]:
                str_note = notes_vocab[track][np.argmax(preds[track][j][0][0][i])]
                duration = durations_vocab[track][np.argmax(preds[track][j][1][0][i])]
                if len(str_note.split(",")) > 1:
                    _chord = chord.Chord(str_note.split(","))
                    _chord.quarterLength = duration
                    y_test_stream[track].append(_chord)
                else:
                    if str_note != "rest":
                        _note = note.Note(str_note)
                        _note.quarterLength = duration
                        y_test_stream[track].append(_note)
                    else:
                        _rest = note.Rest()
                        _rest.quarterLength = duration
                        y_test_stream[track].append(_rest)
    y_test_stream[track].insert(0, instru)

In [13]:
# play seed
x_full_score = stream.Score()
for track, _ in enumerate(tracks):
    p = stream.Part()
    p.append(x_test_stream[track])
    x_full_score.insert(0, p)
x_full_score.makeMeasures()#.show('musicxml')
x_full_score.chordify()
x_full_score.write("midi", "x_test.mid")
sp = midi.realtime.StreamPlayer(x_full_score)
sp.play()

In [443]:
# play generated music
y_full_score = stream.Score()
for track, _ in enumerate(tracks):
    p = stream.Part()
    p.append(y_test_stream[track])
    y_full_score.insert(0, p)
y_full_score.makeMeasures()#.show('musicxml')
y_full_score.chordify()
y_full_score.write("midi", "x_test.mid")
sp = midi.realtime.StreamPlayer(y_full_score)
sp.play()